In [97]:
import matplotlib.pyplot as plt

def init_board():
  return jnp.zeros((4, 4), dtype=int)

def flip_board(board):
  return -board

def valid_mask(board):
  return board == 0

#Always assume action is made by player 1
def next_state(board, action):
  return board.flatten().at[action].add(1).reshape(board.shape)

def sample_action(action_dist, rng):
  action_dist = action_dist.flatten() / action_dist.sum()
  return jax.random.choice(rng, jnp.arange(action_dist.shape[0]), p=action_dist)

def disp_board(board):
  plt.imshow(board)

reward_conv = nn.Conv(features=4, kernel_size=(3, 3), use_bias=False, padding='SAME')
stripe_filter = jnp.array([[0, 0, 0], [1, 1, 1], [0, 0, 0]])
reward_kernel = jnp.expand_dims(jnp.stack([jnp.eye(3), jnp.eye(3)[::-1, :], stripe_filter, stripe_filter.T], axis=2), 2)

conv_param = {'params': {'kernel': reward_kernel}}
# print(jnp.expand_dims(jnp.eye(4), (0, 3)))

def get_reward(board, conv_param):
    board_score = reward_conv.apply(conv_param, jnp.expand_dims(board, (0, 3)))
    # print(jnp.transpose(board_score[0]))
    is_win = (jnp.max(board_score) >= 3).astype(int)
    is_loss = (jnp.min(board_score) <= -3).astype(int)
    return is_win - is_loss

In [98]:
board = init_board()

rng = jax.random.PRNGKey(40)
for i in range(16):
  _, rng = jax.random.split(rng, 2)
  action_dist = jnp.ones(board.shape) * valid_mask(board)
  next_action = sample_action(action_dist, rng)
  board = flip_board(next_state(board, next_action))
  reward = get_reward(board, conv_param)
  print(reward)
  if(reward != 0):
    break

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
-1


In [100]:
# board = board.at[3, 0].set(0)
# print(board)

get_reward(-jnp.eye(4, dtype=int), conv_param)

DeviceArray(-1, dtype=int32)

In [32]:
import jax
import jax.numpy as jnp                # JAX NumPy

from flax import linen as nn           # The Linen API
from flax import traverse_util
from flax.training import train_state  # Useful dataclass to keep train state

import numpy as np                     # Ordinary NumPy
import optax                           # Optimizers



In [ ]:
class Node:
  def __init__(self, prior):
      self.prior = prior      # The prior probability of selecting this state from its parent

      self.children = {}      # A lookup of legal child positions
      self.visit_count = 0    # Number of times this state was visited during MCTS. "Good" are visited more then "bad" states.
      self.value_sum = 0      # The total value of this state from all visits
      self.state = None       # The board state as this node

  def value(self):
       # Average value for a node
      return self.value_sum / self.visit_count

def run(self, state, num_simulations=5):
    root = Node(0)
    # EXPAND root
    root.expand(self.model, self.game, state)

    for _ in range(num_simulations):
        node = root
        search_path = [node]
        # SELECT
        while node.expanded():
            action, node = node.select_child()
            search_path.append(node)

        parent = search_path[-2]                                                  
        state = parent.state                                                      
        next_state, _ = self.game.get_next_state(state, action=action) 
        value = self.game.get_reward_for_player(next_state, player=1)
        if value is None:
            # EXPAND
            value = node.expand(self.model, self.game, flip_board(next_state))

        self.backup(search_path, value, parent.to_play * -1)

    return root

In [ ]:

STATE_DIM = 16 #Dimension of 4x4 tic-tac-toe board
ACTION_DIM = 16
MAX_SIZE = 1e6
C_BASE, C_INIT = 1.0, 1.0
class MCTS:
    def __init__(self):
        self.state = np.zeros((STATE_DIM, MAX_SIZE))
        self.state_lookup = {} #Maps state representation to index

        self.value = np.zeros(MAX_SIZE)
        self.visit_count = np.zeros(MAX_SIZE)
        self.total_value = np.zeros(MAX_SIZE)
        
        self.action_visits  = np.zeros(MAX_SIZE, ACTION_DIM)
        self.action_total_value = np.zeros(MAX_SIZE, ACTION_DIM)
        self.action_mean_value = np.zeros(MAX_SIZE, ACTION_DIM)
        self.action_prior = np.zeros(MAX_SIZE, ACTION_DIM)
    
    #Assumes state is already expanded, and uses MCTS info to pick best action
    def select_action(self, state_index):
        state_visits = self.visit_count[state_index]
        exp_rate = np.log((1+state_visits + C_BASE)/C_BASE) + C_INIT
        model_prior = self.action_prior[state_index]
        sa_visits = self.action_visits[state_index]
        sa_mean_value = self.action_mean_value[state_index]
        action_distr = sa_mean_value + exp_rate*np.sqrt(state_visits)*model_prior/(1+sa_visits)
        return np.argmax(action_distr)

    def search(self, state, model):
    
    def search_batch(self, state, model):

    def est_action_distr(self, state, model):


        

In [31]:


rng = jax.random.PRNGKey(42)
rng, inp_rng, init_rng = jax.random.split(rng, 3)
inp = jnp.zeros((10, 10, 1))

# params = conv.init(init_rng, inp)
# print(jax.tree_util.tree_map(jnp.shape, params))




(10, 10, 3)

In [ ]:
class CNN(nn.Module):
  """A simple CNN model."""

  @nn.compact
  def __call__(self, x):
    x = nn.Conv(features=32, kernel_size=(3, 3))(x)
    return x

cnn = CNN()
params = cnn.init(rng, jnp.ones([1, 28, 28, 1]))['params']